by [@tozCSS](https://twitter.com/tozCSS)

This notebook contains some maps and plots regarding the last two general election results in Turkey (held in Jun 7, 2015 and Nov 1, 2015).

## Data Source
Both Jun 7 and Nov 1 election results are scraped from the Nov 1, 2015 [election results page]('http://www.yenisafak.com/secim-2015-kasim/secim-sonuclari') of the Yenisafak daily. You can find them as CSV files in the data folder.

## Election Results Maps
I created two maps for the last two general election results of Turkey. If you click on any of the cities in those maps (below),
vote shares of the four parties are shown for that particular election.

The colors in these maps represent the vote share of AKP in those cities. Since one of the more interesting questions regarding the Nov 1, 2015 election is in which cities AKP increased its votes, I created density maps to facilitate us seeing them.

## Vote Share Scatter Plots of Each Party per City for the Last Two Elections
For all of the parties, their vote shares in the last two elections is compared in a scatter plot where the dots represent the cities and their colors represent the geographic region they belong to. One interesting pattern for example is that unlike other parties, CHP increased its vote share in some cities and lost some in others. On the other hand we see that AKP increased its vote share in every city, whereas MHP and HDP lost their shares almost in every single city.

In [1]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
import folium
from IPython.display import HTML
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
# scrape both Nov 1st and Jun 7th election results at  city level
resource = requests.get('http://www.yenisafak.com/secim-2015-kasim/secim-sonuclari')
soup = BeautifulSoup(resource.content.decode('utf-8','ignore'), "lxml")
# kill all script, style, sub, sup and b elements
for script in soup(["script", "style", "sup", "sub", "b"]):
    script.extract()    # rip out some visualization elements
cities = soup.find_all(class_='data  compare')[3:] #81 city results
nov = pd.DataFrame(columns=('city','AKP', 'CHP','MHP','HDP','others'));nov.name = 'nov'
jun = pd.DataFrame(columns=('city','AKP', 'CHP','MHP','HDP','others'));jun.name = 'jun'
for city in cities:
    info = city.find_all('span')
    nov1 = [float(info[i].text.replace(',','.')) for i in range(16,21)] #Nov 1st results
    jun7 = [float(info[i].text.replace(',','.')) for i in range(23,28)] #Jun 7th results
    nov.loc[len(nov)]=[info[0].text] + nov1 #add city name and insert row to the dataframe
    jun.loc[len(jun)]=[info[0].text] + jun7 #add city name and insert row to the dataframe
nov.to_csv('data/nov.csv',index=False,encoding='utf-8')
jun.to_csv('data/jun.csv',index=False,encoding='utf-8')

In [3]:
def create_json(df):
    """create geojson files annotated with election results"""
    f = json.load(open('turkey.geojson',encoding='utf-8'))
    parties = ['AKP', 'CHP','MHP','HDP']
    for ft in f['features']:
        city = ft['properties']['name']
        shares = str(df[df.city==city][parties].values).strip('[]').split()
        p = '<b>'+city+'</b><br>'
        for i in range(4):
            p += parties[i]+': '+shares[i]+'%<br>'
        p = p[:-4]
        ft['properties'] = {'name':city,'popupContent':p}

    with open('tr_'+df.name+'.geojson', 'w',encoding='utf-8') as outfile:
        json.dump(f,outfile,ensure_ascii=False)

create_json(nov)
create_json(jun)

In [4]:
# AKP Vote Shares (%), Nov 1, 2015 Turkish General Elections
mapname = 'akp_share_nov'
tr_geo = 'tr_nov.geojson'
tr = folium.Map(location=[39.5, 35], zoom_start=6, tiles='Mapbox Bright')
tr.geo_json(geo_path=tr_geo, data=nov, data_out=mapname+'.json',
                columns=['city', 'AKP'],
                threshold_scale=[15,25,35,45,55,65],
                key_on='feature.properties.name',
                fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
                legend_name='AKP Vote Shares (%), Nov 1, 2015 Turkish General Elections')
tr.create_map(path=mapname+'.html')
HTML('<iframe src='+mapname+'.html style="width: 100%; height: 500px; border: none"></iframe>')

In [5]:
# AKP Vote Shares (%), Jun 7, 2015 Turkish General Elections
mapname = 'akp_share_jun'
tr_geo = 'tr_jun.geojson'
tr = folium.Map(location=[39.5, 35], zoom_start=6, tiles='Mapbox Bright')
tr.geo_json(geo_path=tr_geo, data=jun, data_out=mapname+'.json',
                columns=['city', 'AKP'],
                threshold_scale=[15,25,35,45,55,65],
                key_on='feature.properties.name',
                fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
                legend_name='AKP Vote Shares (%), Jun 7, 2015 Turkish General Elections')
tr.create_map(path=mapname+'.html')
HTML('<iframe src='+mapname+'.html style="width: 100%; height: 500px; border: none"></iframe>')

In [6]:
#Scatter plot
import plotly.plotly as py
from plotly.graph_objs import *
from palettable.colorbrewer.qualitative import Dark2_7 as colmap

il = pd.read_csv('data/city_meta.csv',usecols=['il','bolge'])
colors = dict(zip(il['bolge'].unique().tolist(),colmap.hex_colors))
parties = ['AKP', 'CHP','MHP','HDP']
jun['bolge'] = il['bolge']
nov['bolge'] = il['bolge']

for p in parties:
    lim = max(nov[p].max(),jun[p].max())
    #each region is a trace, otherwise they do not show up in the legend
    traces = [Scatter(x=jun[jun.bolge==b][p],y=nov[nov.bolge==b][p],
                      mode='markers', text=il[il.bolge==b]['il'], name=b,
                      marker=Marker(color=v)) for b,v in colors.items()]
    traces.append(Scatter(x=[0,lim],y=[0,lim],mode='lines', name='y=x',
                        line=Line(color='grey',dash='dash')))
    data = Data(traces)
    layout = Layout(title=p+' Nov 1, 2015 vs Jun 7, 2015 Vote Shares',
                    autosize=True,
                    xaxis=XAxis(title=p+" Jun 7, 2015 Vote Shares",zeroline=False),
                    yaxis=YAxis(title=p+" Nov 1, 2015 Vote Shares",zeroline=False),
                    legend=Legend(x=.01,y=1))
    fig = Figure(data=data,layout=layout)
    url = py.plot(fig,filename= p+' Nov 1, 2015 vs Jun 7, 2015 Vote Shares')

In [7]:
HTML("""<div><a href="https://plot.ly/~toz/828/" target="_blank" title="AKP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="display: block; text-align: center;"><img src="https://plot.ly/~toz/828.png" alt="AKP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a><script data-plotly="toz:828"  src="https://plot.ly/embed.js" async></script></div>""")

In [8]:
HTML("""<div><a href="https://plot.ly/~toz/830/" target="_blank" title="CHP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="display: block; text-align: center;"><img src="https://plot.ly/~toz/830.png" alt="CHP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a><script data-plotly="toz:830"  src="https://plot.ly/embed.js" async></script></div>""")

In [9]:
HTML("""<div><a href="https://plot.ly/~toz/832/" target="_blank" title="MHP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="display: block; text-align: center;"><img src="https://plot.ly/~toz/832.png" alt="MHP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a><script data-plotly="toz:832"  src="https://plot.ly/embed.js" async></script></div>""")

In [10]:
HTML("""<div><a href="https://plot.ly/~toz/834/" target="_blank" title="HDP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="display: block; text-align: center;"><img src="https://plot.ly/~toz/834.png" alt="HDP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a><script data-plotly="toz:834"  src="https://plot.ly/embed.js" async></script></div>""")